In [ ]:
import pandas as pd
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta 
import mydashtools as dt
import plotly.express as px
import hashlib
pd.options.plotting.backend = "plotly"
if dt.is_notebook():
    __file__ = "notebook/"

services_baristas_annule = pd.read_csv(os.path.dirname(os.path.realpath(__file__))+"/../calendar/baristas.csv", sep=";",decimal=',',low_memory=False, index_col=False)
services_baristas_annule['Mois'] = pd.to_datetime(services_baristas_annule['date'].apply(lambda x: x[0:7]+'-01'))
services_baristas_annule['Date'] = pd.to_datetime(services_baristas_annule['date'])
del services_baristas_annule['date']

services_baristas_annule['nom'] = services_baristas_annule['nom'].str.title()
t2n = services_baristas_annule.groupby('telephone')['nom'].first().to_dict()
n2t = services_baristas_annule.groupby('nom')['telephone'].first().to_dict()
services_baristas_annule['telephone_nom'] = services_baristas_annule['nom'].map(n2t)
services_baristas_annule['telephone'] = services_baristas_annule['telephone_nom'].combine_first(services_baristas_annule['telephone']).fillna('')
services_baristas_annule['nom_telephone'] = services_baristas_annule['telephone'].map(t2n)
services_baristas_annule['nom'] = services_baristas_annule['nom_telephone'].combine_first(services_baristas_annule['nom'])
services_baristas_annule['nom_tel'] = services_baristas_annule['nom'].fillna('') + services_baristas_annule['telephone'].fillna('')
services_baristas_annule['barista_hash'] = services_baristas_annule['nom_tel'].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
del services_baristas_annule['nom_telephone']
del services_baristas_annule['telephone_nom']
del services_baristas_annule['nom_tel']

In [ ]:
creneaux_annules = services_baristas_annule[services_baristas_annule['role'] == "CRENEAU ANNULE"][['Mois', 'role']].groupby('Mois').count()
creneaux_annules['Année'] = creneaux_annules.index.year
creneaux_annules = creneaux_annules.reset_index().set_index(['Mois', 'Année']).unstack('Année').fillna(0)
creneaux_annules.columns = creneaux_annules.columns.droplevel(0)

fig = px.bar(creneaux_annules)
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'baristas_01_creneaux_annules')

In [ ]:
import hashlib

services_baristas = services_baristas_annule[services_baristas_annule['role'] != "CRENEAU ANNULE"]
services_baristas = services_baristas_annule[services_baristas_annule['role'] != "RENFORT"]

premier_service = services_baristas.groupby('barista_hash')[['Date']].min()['Date'].to_dict()
services_baristas['Date premier service'] = services_baristas['barista_hash'].map(premier_service)
services_baristas['is premier service'] = services_baristas['Date'] == services_baristas['Date premier service']

dernier_service = services_baristas.groupby('barista_hash')[['Date']].max()['Date'].to_dict()
services_baristas['Date dernier service'] = services_baristas['barista_hash'].map(dernier_service)
services_baristas['is dernier service'] = services_baristas['Date'] == services_baristas['Date dernier service']

In [ ]:
nb_baristas = services_baristas.groupby(['Mois','nom','telephone'])[['role']].count().rename(columns= {'role': 'nb'})
nb_baristas = nb_baristas.groupby('Mois').aggregate(func=['count', 'sum'])
nb_baristas.columns = nb_baristas.columns.droplevel(0)
nb_baristas = nb_baristas.rename(columns={'count': 'Nb de baristas', 'sum': 'Nb de services réalisés'})
nb_baristas['Année'] = nb_baristas.index.year
nb_baristas = nb_baristas.reset_index().set_index(['Mois', 'Année']).unstack('Année').fillna(0)

In [ ]:
fig = px.bar(nb_baristas[['Nb de baristas']].stack('Année').groupby('Année').sum())
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'baristas_02_nb_baristas_par_an')

In [ ]:
fig = px.bar(nb_baristas[['Nb de services réalisés']].stack('Année').groupby('Année').sum())
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
fig.update_layout(separators=', ')
fig.update_yaxes(tickformat=",d")
dt.fig_save_or_show(fig, 'baristas_03_nb_services_realises_par_an')

In [ ]:
fig = px.bar(nb_baristas['Nb de baristas'])
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'baristas_04_nb_baristas_par_mois')

In [ ]:
fig = px.bar(nb_baristas['Nb de services réalisés'])
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'baristas_05_nb_services_réalisés_par_mois')

In [ ]:
services = services_baristas[services_baristas['role'] == "REFERENT"].groupby(['Mois','nom','telephone'])[['role']].count().rename(columns= {'role': 'nb'})
services = services.groupby('Mois').aggregate(func=['count', 'sum'])
services.columns = services.columns.droplevel(0)
services['Année'] = services.index.year

services = services.reset_index().set_index(['Mois', 'Année']).unstack('Année').fillna(0)

In [ ]:
fig = px.bar(services['count'])
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'baristas_06_nb_referents_par_mois')

In [ ]:
nb_services = services_baristas[['Mois', 'Date', 'role']].groupby(['Mois', 'Date']).count().rename(columns={'role': 'nb'}).reset_index()[['Mois', 'Date']].groupby('Mois').count()
nb_services['Année'] = nb_services.index.year
nb_services = nb_services.reset_index().set_index(['Mois', 'Année']).unstack('Année').fillna(0)
nb_services.columns = nb_services.columns.droplevel(0)
nb_services

fig = px.bar(nb_services)
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
dt.fig_save_or_show(fig, 'baristas_07_nb_services_ouverts')

In [ ]:
limit_date_year = datetime.now() + relativedelta(years=-1)
services_baristas_year = services_baristas[services_baristas['Mois'] > limit_date_year]
baristas_year = services_baristas_year.groupby(['nom', 'barista_hash'])[['role']].count().rename(columns={'role': 'nb'})
baristas_year.sort_values(by='nb', ascending=False, inplace=True)
baristas_year.rename(columns={'nb': 'Nb de services'}, inplace=True)
baristas_year = baristas_year.reset_index().set_index('nom')

fig = px.bar(baristas_year['Nb de services'])
fig.update_layout(showlegend=False,xaxis_title=None,yaxis_title=None)
fig.update_xaxes(showticklabels=False)
dt.fig_save_or_show(fig, 'baristas_08_nb_services_par_baristas_12_mois')

In [ ]:
if dt.is_notebook():
    print(baristas_year.head(60))

In [ ]:
res={
    'un_seul_service': len(baristas_year[baristas_year['Nb de services'] < 2].dropna().reset_index()['nom'].unique()),
    'plus_de_20_services': len(baristas_year[baristas_year['Nb de services'] > 20].dropna().reset_index()['nom'].unique()),
    'plus_de_40_services': len(baristas_year[baristas_year['Nb de services'] > 40].dropna().reset_index()['nom'].unique()),
    'plus_de_60_services': len(baristas_year[baristas_year['Nb de services'] > 60].dropna().reset_index()['nom'].unique()),
    'nb_baristas': len(baristas_year.dropna().reset_index()['nom'].unique()),
    'moyenne_du_nb_de_services': baristas_year['Nb de services'].mean()
}
baristas_std = baristas_year.reset_index()
res['ecart_type_du_nb_de_services'] = baristas_std[baristas_std.index == int(len(baristas_std)/2)].reset_index()['Nb de services'][0]
if dt.is_notebook():
    print(res)

In [ ]:
services_baristas_year['has_premier_service'] = services_baristas_year['Date'].isin(services_baristas_year[services_baristas_year['is premier service']]['Date'].to_list())
t2nb = baristas_year.set_index('barista_hash')['Nb de services'].to_dict()
services_baristas_year['nb services'] = services_baristas_year['barista_hash'].map(t2nb)
services_baristas_year['is_unique_service'] = services_baristas_year['nb services'] == 1
has_premier_service_year = services_baristas_year[services_baristas_year['has_premier_service']]
n2nb1 = has_premier_service_year.groupby('nom')['has_premier_service'].count().to_dict()
has_premier_service_year['nb 1er services'] = has_premier_service_year['nom'].map(n2nb1)
has_premier_service = has_premier_service_year[has_premier_service_year['Date'].isin(has_premier_service_year[has_premier_service_year['is_unique_service']]['Date'])]
has_premier_service = has_premier_service.groupby(['nom', 'role'])['nb 1er services'].aggregate(func=['first', 'count']).rename(columns={'first': 'nb avec 1er service', 'count': 'nb avec uniq service'})
has_premier_service['pc'] = has_premier_service['nb avec uniq service'] * 100 / has_premier_service['nb avec 1er service']
has_premier_service = has_premier_service[has_premier_service['nb avec uniq service'] != 1]
has_premier_service.sort_values(by='pc', ascending=False)